In [1]:
import nltk
import pandas as pd

In [2]:
training_data=pd.read_csv('training_twitter_x_y_train.csv')
x_test=pd.read_csv('test_twitter_x_test.csv')

In [3]:
x_test.shape

(3660, 11)

In [4]:
sentiments=training_data.airline_sentiment
text=training_data.text
documents=[]
for i in range(len(sentiments)):
    documents.append((text[i],sentiments[i]))

In [5]:
from nltk.corpus import stopwords
from string import punctuation

In [7]:
s=stopwords.words('english')
p=list(punctuation)
stop_words=p+s
len(stop_words)

211

In [8]:
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

In [9]:
lemmatizer=WordNetLemmatizer()

In [10]:
def simplify_pos_tag(tag):
    if tag.startswith('N'):
        return 'n'
    if tag.startswith('J'):
        return 'a'
    if tag.startswith('R'):
        return 'r'
    if tag.startswith('V'):
        return 'v'
    return 'n'

In [11]:
import re

In [12]:
def hasNumbers(text):
    return bool(re.search('[0-9]+',text))

In [13]:
def clean_text(text):
    result=[]
    for word in text.split():
        word=word.strip(punctuation)
        if ((word.lower() not in stop_words) and (not hasNumbers(word.lower())) and (len(word.lower())>3)):
            posTag=pos_tag([word.lower()])[0][1]
            lemmatized_word=lemmatizer.lemmatize(word.lower(),simplify_pos_tag(posTag))
            result.append(lemmatized_word)
    return result

In [14]:
documents_modified=[(clean_text(text),sentiment) for text,sentiment in documents]
x_test=[clean_text(text) for text in x_test]


In [16]:
import numpy as np

In [17]:
X_train = [" ".join(text) for text,sentiment in documents_modified]
Y_train = [sentiment for text,sentiment in documents_modified]
X_test = [" ".join(text) for text in x_test]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
from sklearn.naive_bayes import MultinomialNB

In [20]:
cv=CountVectorizer(max_features=450000,ngram_range=(1,2))
X_train_features = cv.fit_transform(X_train)
X_test_features = cv.transform(X_test)
clf=MultinomialNB()
clf.fit(X_train_features,Y_train)
clf.predict(X_test_features)

array(['negative', 'negative', 'negative', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'negative'], dtype='<U8')

In [26]:
clf.score(X_test_features,Y_pred)

1.0

In [22]:
Y_pred=clf.predict(X_test_features)

In [25]:
np.savetxt('Twitter Sentiment Analysis12.csv',Y_pred,fmt="%s")

In [31]:
X_train_features.shape

(10980, 62487)

In [32]:
Y_pred.shape

(11,)